In [ ]:
import pandas as pd
import re

# Example DataFrame
df = pd.DataFrame({
    'my_column': ['A;B', 'B;C', 'A', 'C;D', 'B', 'M;N', 'O;P', 'A;M', 'a-b; m n', None],
    'customer_number': ['123.0', '456.0', '789', 'abc', '12 3', '45-6', '78%9', '?', None, '001.0']
})

# Split the column by ';' into lists
df['my_column_split'] = df['my_column'].str.split(';')

# Clean values: strip, capitalize, replace spaces and hyphens with underscores
def clean_list(lst):
    if not isinstance(lst, list):
        return []
    return [x.strip().upper().replace(' ', '_').replace('-', '_') for x in lst]

df['my_column_split'] = df['my_column_split'].apply(clean_list)

# Clean customer_number field
def clean_customer_number(val):
    if pd.isnull(val):
        return ''
    val = str(val).strip()
    if val.endswith('.0'):
        val = val[:-2]
    val = re.sub(r'[^A-Za-z0-9_]', '_', val)  # Replace unwanted chars with underscore
    return val

df['customer_number_clean'] = df['customer_number'].apply(clean_customer_number)

# Explode the lists into rows
exploded = df.explode('my_column_split')

# Create binary columns for each unique value
dummies = pd.get_dummies(exploded['my_column_split'])

# Group back to original rows by max (if any value is 1, keep 1)
binary_df = dummies.groupby(exploded.index).max()

# Create business and savings flags
business_flags = {'A', 'B', 'C', 'D', 'E', 'X', 'F'}
savings_flags = {'M', 'N', 'O', 'P'}

binary_df['business'] = binary_df[list(business_flags & set(binary_df.columns))].any(axis=1).astype(int)
binary_df['savings'] = binary_df[list(savings_flags & set(binary_df.columns))].any(axis=1).astype(int)

# Concatenate with original DataFrame if needed
result = pd.concat([df, binary_df], axis=1)

result

In [ ]:
# ...existing code...

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Load your Salesforce data (replace with your file path or DataFrame) ---
# df = pd.read_csv('salesforce_opportunities.csv', parse_dates=['Opportunity_Created'])

# Use the binary_df from your first block (already created)
# Merge Opportunity_Created and Customer_ID into binary_df if needed
binary_df['Opportunity_Created'] = df['Opportunity_Created']
binary_df['Customer_ID'] = df['Customer_ID']

# --- Step 1: Get first interaction per customer ---
binary_df['First_Interaction'] = binary_df.groupby('Customer_ID')['Opportunity_Created'].transform('min')

# --- Step 2: Calculate months since first interaction ---
binary_df['Months_Since_First_Interaction'] = (
    (binary_df['Opportunity_Created'] - binary_df['First_Interaction']) / pd.Timedelta(days=30)
).round().astype(int)

# --- Step 3: Melt product columns ---
product_cols = [col for col in binary_df.columns if col not in [
    'Opportunity_Created', 'Customer_ID', 'First_Interaction', 'Months_Since_First_Interaction', 'business', 'savings'
]]

melted = binary_df.melt(
    id_vars=['Customer_ID', 'Opportunity_Created', 'First_Interaction', 'Months_Since_First_Interaction'],
    value_vars=product_cols,
    var_name='Product_Name',
    value_name='Discussed'
)

# --- Step 4: Remove rows where no products were discussed ---
melted = melted[melted['Discussed'] == 1]

# --- Step 5: Aggregate product inquiries by lifecycle stage ---
product_timeline = (
    melted.groupby(['Product_Name', 'Months_Since_First_Interaction'])
    .size()
    .reset_index(name='Inquiry_Count')
)

# --- Step 6: Pivot for visualization ---
timeline_pivot = product_timeline.pivot(
    index='Months_Since_First_Interaction',
    columns='Product_Name',
    values='Inquiry_Count'
).fillna(0)

# --- Step 7: Plot heatmap ---
plt.figure(figsize=(14, 6))
sns.heatmap(timeline_pivot, cmap='YlGnBu', linewidths=0.5, annot=True, fmt='.0f')
plt.title('Customer Product Inquiries Over Lifecycle')
plt.xlabel('Product')
plt.ylabel('Months Since First Interaction')
plt.tight_layout()
plt.show()

# ...existing code...